## Import libraries and df

In [ ]:
import csv
import pandas as pd
import numpy as np

import random
import os , glob
import os.path

from shutil import copyfile
from os import path
from pathlib import Path



In [ ]:
root_path = "..\\data\\"

df_path = root_path + "full_data_description.csv"

df = pd.read_csv(df_path)

### Filter only good claims (where the transcript is matched the cards that the player claimed to place)

In [ ]:
good_claims = df["MATCH(ASR,CLAIM TEXT)"] == True
df = df[good_claims]
df

## Split train-test
 - By subjetcs
 - 70% - train , 30% - test 
 - Split each task to support and query sets

### Define division

In [ ]:
TRAIN_SIZE = round(len(df)*0.7)
TEST_SIZE = len(df)-TRAIN_SIZE

print("train size: {} \ntest size: {}".format(TRAIN_SIZE , TEST_SIZE))
print("Amount of claims:" ,len(df))
amount_of_claims_per_w = df.groupby('WorkerId').size()

In [ ]:
np.random.seed(2210)
counter_train = 0
counter_test = 0
train_w = []
test_w = []
for w in amount_of_claims_per_w.sample(frac=1).iteritems():
    if counter_train + w[1] <= TRAIN_SIZE:
        counter_train += w[1]
        train_w.append(w[0])
    else:
        counter_test += w[1]
        test_w.append(w[0])
print("train size: {} \ntest size: {}".format(counter_train , counter_test))
print("train workers: {} \ntest workers: {}".format(len(train_w) , len(test_w)))





### Remove subjects with less than 2 claims per class 

- Check amount of claims per subject
- Save updated results


In [ ]:
change_idx  = df.index[df['WorkerId'] == '------'].tolist()
temp_df =df
for i in change_idx:
    temp_df.loc[i,['WorkerId']] = df.loc[i,'name']


temp_df['IsFalseClaim'] = 1-temp_df['IsTrueClaim'] 
filtered_workers=pd.pivot_table(temp_df, index=['WorkerId'],values=['IsTrueClaim','IsFalseClaim'],aggfunc=np.sum)
filtered_workers = filtered_workers[filtered_workers['IsFalseClaim']>1][filtered_workers['IsTrueClaim']>1]
# print(filtered_workers.sort_values(by=(['IsTrueClaim','IsFalseClaim'])[:]).to_string())
# pd.set_option('display.max_rows', None)
display(filtered_workers.sort_values(by=(['WorkerId'])[:]))


In [ ]:
if '------' in train_w:
    def_set = 'train'
else:
    def_set = 'test'  
for worker in filtered_workers.index:
    if worker in train_w:
        filtered_workers.at[worker,'set']='train'
    elif worker in test_w:
        filtered_workers.at[worker,'set']='test'
    else:
        filtered_workers
        filtered_workers.at[worker,'set']=def_set
    
    # A bug in original division
    try:
        filtered_workers.drop(train_w[0],inplace=True)
        filtered_workers.drop(test_w[0],inplace=True)
    except:
        ""       
tr_list =filtered_workers.index[filtered_workers['set'] == 'train'].tolist()
ts_list = filtered_workers.index[filtered_workers['set'] == 'test'].tolist()
workers_list = tr_list+ts_list
print("Amount of train workers = {}\nAmount of test workers = {}".format(len(tr_list),len(ts_list)))


### Save all filtered subject details to df

In [ ]:
all_claims_df = df
all_claims_df = all_claims_df.loc[all_claims_df['WorkerId'].isin(workers_list)]
all_claims_df['path'] = all_claims_df['name']+'/'+all_claims_df['file_name']
all_claims_df.to_csv(root_path+"all_sampels_by_tasks\\samples_description.csv")
all_claims_df

### Copy files to folder by train-test - > subject -> true/false

In [ ]:
division_path = root_path + "train_test_division\\"
os.mkdir(division_path)
train_dir = division_path + "train\\"
os.mkdir(train_dir)
test_dir = division_path + "test\\"
os.mkdir(test_dir)
counterF_tr = 0
counterT_tr = 0
counterN_tr = 0
counterF_ts = 0
counterT_ts = 0

is_train = 0

paths = df.path
names = df.file_name
subjects =df.WorkerId
claims = df.IsTrueClaim

for claim_path, name, w, claim in zip(paths, names, subjects,claims):
        if w in tr_list: # A train subject
            if not path.isdir(train_dir+w):
                os.mkdir(train_dir+w)
            is_train = 1
            if not path.isdir(train_dir+w+"\\true_claims\\"):
                os.mkdir(train_dir+w+"\\true_claims\\")
                os.mkdir(train_dir+w+"\\false_claims\\")

        elif w in ts_list: # A test subject
            if not path.isdir(test_dir+w):
                os.mkdir(test_dir+w)
            is_train = 0
            if not path.isdir(test_dir+w+"\\true_claims\\"):
                os.mkdir(test_dir+w+"\\true_claims\\")
                os.mkdir(test_dir+w+"\\false_claims\\")
        
        else: # '------' worker Id
            is_train = -1
        if is_train == 1:
            if not path.isdir(train_dir+w):
                os.mkdir(train_dir+w)
            if claim:
                counterT_tr += 1
                copyfile(claim_path,train_dir+w+"\\true_claims\\" + name)
            elif not claim:
                counterF_tr += 1
                copyfile(claim_path, train_dir+w+"\\false_claims\\" + name)
        elif is_train == 0:
            if not path.isdir(test_dir+w):
                os.mkdir(test_dir+w)
            if claim:
                counterT_ts += 1
                copyfile(claim_path, test_dir+w+"\\true_claims\\" + name)
            elif not claim:
                counterF_ts += 1
                copyfile(claim_path, test_dir+w+"\\false_claims\\" + name)

print("TRAIN:\ntrue: {} ,false: {}, {}".format(counterT_tr,counterF_tr,counterF_tr/(counterT_tr+counterF_tr)))
print("TEST:\ntrue: {} ,false: {}, {}".format(counterT_ts,counterF_ts,counterF_ts/(counterT_ts+counterF_ts)))

print("\nTRUE: {}\nFALSE: {}".format(counterT_tr +counterT_ts ,counterF_tr,counterF_ts))



In [ ]:
division_path = root_path + "train_test_division\\"
train_dir = division_path + "train\\"
test_dir = division_path + "test\\"


def create_df(directory,type=""):
    files = []
    paths = []
    workers = []
    labels = []
    for dir_worker in os.listdir(directory): # Worker

        path_worker = directory+"\\"+dir_worker+"\\"
        if os.path.isdir(path_worker):
                for dir_type in os.listdir(path_worker): # true/false
                    path_type = path_worker + "\\" + dir_type + "\\"
                    if dir_type == "false_claims":
                        label = 1
                    else:
                        label = 0
                    for file in os.listdir(path_type):
                        file_path =Path(path_type + "\\" + file)
                        
                        files.append(file)
                        paths.append(file_path)
                        workers.append(dir_worker)
                        labels.append(label)

    df = pd.DataFrame(list(zip(files, paths, workers, labels)),
                columns =['file_name', 'path', 'task','label'])
    df.to_csv( root_path+type+"_df.csv",index = False)
    return df

train_df = create_df(train_dir,"train")
test_df = create_df(test_dir,"test")

### Split each subject to support and query sets

In [ ]:

paths = df.path
names = df.file_name
workers =df.WorkerId
claims = df.IsTrueClaim


def sample_files (pathlist):

    # list all files in dir
    files = [pathlist+file for file in os.listdir(pathlist)]
    # select 2 of the files randomly 
    random_files = np.random.choice(files, 2)

    return random_files

def split(directory):

    for dir in os.listdir(directory):
        f = os.path.join(directory, dir)
        if os.path.isdir(f):
            if not path.isdir(f+"\\support\\"):
                os.mkdir(f+"\\support\\")
            if not path.isdir(f+"\\query\\"):
                os.mkdir(f+"\\query\\")
            if not path.isdir(f+"\\support\\true_claims"):
                os.mkdir(f+"\\support\\true_claims")
            if not path.isdir(f+"\\support\\false_claims"):
                os.mkdir(f+"\\support\\false_claims")
            if not path.isdir(f+"\\query\\true_claims"):
                os.mkdir(f+"\\query\\true_claims")
            if not path.isdir(f+"\\query\\false_claims"):
                os.mkdir(f+"\\query\\false_claims")

            # filelist = glob.glob(os.path.join(f+"\\support\\", "*/*.wav"))
            # for f in filelist:
            #     os.remove(f)
        
            true_path = f+"\\true_claims\\"
            false_path = f+"\\false_claims\\"

            # Sample 2 files from true and from false and move it to the support dir.
            support_files = sample_files(false_path).tolist()
            support_files+=(sample_files(true_path).tolist())

            for file in support_files:
                dest_path = f + "\\support\\"+file.split("\\")[-2]+"\\"+file.split("\\")[-1]
                print(file,dest_path)
                if os.path.isfile(file):
                    ""
                    os.replace(file, dest_path)

            # Move rest of the files to query folder and delete the original folders
            for file in os.listdir(true_path):
                dest_path = f + "\\query\\true_claims"+"\\"+file
                if os.path.isfile(true_path+"\\"+file):
                    os.replace(true_path+"\\"+file, dest_path)

            for file in os.listdir(false_path):
                dest_path = f +  "\\query\\false_claims"+"\\"+file
                if os.path.isfile(false_path+"\\"+file):
                    os.replace(false_path+"\\"+file, dest_path)
            
            os.rmdir(true_path)
            os.rmdir(false_path)



split(train_dir)
split(test_dir)

  
  

### Create df of train and test division

[file, path , worker(task), label, set (support/ query)]

In [ ]:
division_path = root_path + "train_test_division\\"
train_dir = division_path + "train\\"
test_dir = division_path + "test\\"

def create_df(directory,type=""):
    files = []
    paths = []
    workers = []
    labels = []
    sets = []
    for dir_worker in os.listdir(directory): # Worker

        path_worker = directory+"\\"+dir_worker+"\\"
        if os.path.isdir(path_worker):
            for dir_set in os.listdir(path_worker): #set 
                path_set = path_worker + "\\" + dir_set + "\\"
                for dir_type in os.listdir(path_set): # true/false
                    path_type = path_set + "\\" + dir_type + "\\"
                    if dir_type == "false_claims":
                        label = 1
                    else:
                        label = 0
                    for file in os.listdir(path_type):
                        file_path =Path(path_type + "\\" + file)
                        
                        files.append(file)
                        paths.append(file_path)
                        workers.append(dir_worker)
                        labels.append(label)
                        sets.append(dir_set)

    df = pd.DataFrame(list(zip(files, paths, workers, labels, sets)),
                columns =['file_name', 'path', 'task','label','set'])
    df.to_csv( root_path+type+"_df.csv",index = False)

create_df(train_dir,"train")
create_df(test_dir,"test")


